In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## LOAD DỮ LIỆU

In [2]:
xpath_coco = '../input/cocopersonsegmentation/train2017_new'
ypath_coco = '../input/cocopersonsegmentation/train2017_ann'
x_test_path = '../input/cocopersonsegmentation/val2017_new'
y_test_path = '../input/cocopersonsegmentation/val2017_ann'

In [3]:
input_img_paths_coco = sorted([os.path.join(xpath_coco, fname) for fname in os.listdir(xpath_coco)])
target_img_paths_coco = sorted([os.path.join(ypath_coco, fname) for fname in os.listdir(ypath_coco)])

In [4]:
input_img_val_paths_coco = sorted([os.path.join(x_test_path, fname) for fname in os.listdir(x_test_path)])
target_img_val_paths_coco = sorted([os.path.join(y_test_path, fname) for fname in os.listdir(y_test_path)])

In [5]:
print(f'Độ dài input train: {len(input_img_paths_coco)}, Độ dài target train: {len(target_img_paths_coco)}')
print(f'Độ dài input validation: {len(input_img_val_paths_coco)}, Độ dài target validation: {len(target_img_val_paths_coco)}')

In [6]:
for input_path, target_path in zip(input_img_paths_coco[:4], target_img_paths_coco[:4]):
    print(input_path, "|", target_path)

In [7]:
for input_path, target_path in zip(input_img_val_paths_coco[:4], target_img_val_paths_coco[:4]):
    print(input_path, "|", target_path)

In [8]:
import matplotlib.pyplot as plt

In [9]:
n_images = 5
for i in np.random.randint(0,len(input_img_paths_coco),n_images):
    fig = plt.figure(figsize=(12,6))
    fig.tight_layout()

    plt.subplot(1,2,1)
    img = plt.imread(input_img_paths_coco[i])
    plt.imshow(img)
    plt.title('Image')

    plt.subplot(1,2,2)
    img = plt.imread(target_img_paths_coco[i])
    plt.imshow(img)
    plt.title('Mask')
    plt.show()
    print()

## CHUẨN BỊ DỮ LIỆU TRAINING

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img

In [11]:
class Data_Generator(tf.keras.utils.Sequence):
    def __init__(self,batch_size,img_size,input_img_path,target_img_path):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_path = input_img_path
        self.target_img_path = target_img_path
    def __len__(self):
        return len(self.target_img_path) // self.batch_size
    def __getitem__(self,idx):
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_path[i : i + self.batch_size] 
        batch_target_img_paths = self.target_img_path[i : i + self.batch_size]
        
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")           #(32,256,256,3)
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)                                #(256,256)
            img = np.array(img)/255
            x[j] = img
        
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")             #(32,256,256,1)
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")        #(256,256)
            img = np.array(img)
            img[img!=0] = 1
            y[j] = np.expand_dims(img, 2) 
        return x, y

In [12]:
batch_size = 32
img_size = (256,256)
train_gen = Data_Generator(batch_size, img_size, input_img_paths_coco, target_img_paths_coco)
val_gen = Data_Generator(batch_size, img_size, input_img_val_paths_coco, target_img_val_paths_coco)
x, y = train_gen.__getitem__(0)
print(x.shape, y.shape)

## XÂY DỰNG KIẾN TRÚC MÔ HÌNH

In [13]:
from tensorflow.keras import layers

In [14]:
def downblock(filters, filter_size, previous_layer):
    x = layers.Conv2D(filters, filter_size, padding="same")(previous_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(filters, filter_size, padding="same")(x)
    x = layers.BatchNormalization()(x)

    residual = layers.Conv2D(filters, 1, padding="same")(previous_layer)      #separate layer for addintion
    x = layers.add([x, residual])  # Add back residual

    x = layers.Activation("relu")(x)
    p = layers.MaxPooling2D(2)(x)

    return x,p
 
def bottleneck(filters, filter_size, previous_layer):
    x = layers.Conv2D(filters, filter_size, padding="same")(previous_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(.5)(x)
    x = layers.Conv2D(filters, filter_size, padding="same")(x)

    residual = layers.Conv2D(filters, 1, padding="same")(previous_layer)      #separate layer for addintion
    x = layers.add([x, residual])  # Add back residual

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    return x
 
def upblock(filters, filter_size, previous_layer, layer_to_concat):
    x = layers.Conv2DTranspose(filters, filter_size, strides=2, padding="same")(previous_layer)       #upconvolution
    concat = layers.concatenate([x, layer_to_concat])                                                      #concatenation

    x = layers.Conv2D(filters, filter_size, padding="same")(concat)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(filters, filter_size, padding="same")(x)
    x = layers.BatchNormalization()(x)

    residual = layers.Conv2D(filters, 1, padding="same")(concat)      #separate layer for addintion
    x = layers.add([x, residual])  # Add back residual

    x = layers.Activation("relu")(x)

    return x

In [15]:
input_layer = layers.Input(shape = img_size + (3,))
 
conv1, pool1 = downblock(32, 3, input_layer)
conv2, pool2 = downblock(64, 3, pool1)
conv3, pool3 = downblock(128, 3, pool2)
conv4, pool4 = downblock(256, 3, pool3)
 
conv5 = bottleneck(512,3,pool4)

upconv1 = upblock(256, 3, conv5, conv4) 
upconv2 = upblock(128, 3, upconv1, conv3)
upconv3 = upblock(64, 3, upconv2, conv2)
upconv4 = upblock(32, 3, upconv3, conv1)
 
output_layer = layers.Conv2D(1, 1, padding="same", activation='sigmoid')(upconv4)
model = tf.keras.Model(input_layer, output_layer)
model.summary()

In [16]:
tf.keras.utils.plot_model(model, 'mohinh.jpg', show_shapes=True, show_layer_names=True)

## TRAINING MÔ HÌNH

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss="binary_crossentropy", metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=2)]) 

In [18]:
filepath = "model_epoch_{epoch:00d}_val_loss_{val_loss:03f}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only= False)
callbacks = [checkpoint]

In [19]:
epochs = 1
model_history = model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)

## VẼ TRỰC QUAN LOSS,ACCURACY,IOU

In [27]:
model_history.history

In [28]:
history = {'loss': [0.2582,0.1537,0.1354],
           'accuracy': [0.8925,0.9363,0.9443],
           'mean_io_u': [0.4290,0.4291,0.4288],
           'val_loss': [0.2502,0.2566,0.2920],
           'val_accuracy': [0.8997,0.9046,0.9030],
           'val_mean_io_u': [0.4162,0.4162,0.4162]
}

In [29]:
train_loss = history['loss']
val_loss = history['val_loss']
train_acc = history['accuracy']
val_acc = history['val_accuracy']
train_iou = history['mean_io_u']
val_iou = history['val_mean_io_u']

plt.figure(figsize=(14,6))

plt.subplot(1,3,1)
plt.plot(train_loss, 'r', label='Training loss')
plt.plot(val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.yticks(np.arange(0, .5,.05))
plt.legend()

plt.subplot(1,3,2)
plt.plot(train_acc, 'r', label='Training acc')
plt.plot(val_acc, 'b', label='Validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epoch')
plt.ylabel('acc')
plt.yticks(np.arange(0,1.1,.1))
plt.legend()

plt.subplot(1,3,3)
plt.plot(train_iou, 'r', label='Training mean_io_u')
plt.plot(val_iou, 'b', label='Validation mean_io_u')
plt.title('Training and Validation mean_io_u')
plt.xlabel('Epoch')
plt.ylabel('mean_io_u')
plt.yticks(np.arange(0,1,.05))
plt.legend()
plt.show()

## CHÈN BACKGROUND VÀO

In [42]:
import seaborn as sns
import os,cv2
from skimage import io

In [43]:
def ploting(imgpath, maskpath):
    plt.figure(figsize=(12,4))

    im = io.imread(imgpath)
    im = cv2.resize(im,img_size)
    im = np.array(im)/255

    plt.subplot(1,3,1)
    plt.title('Original')
    plt.imshow(im)

    im = im.reshape((1,)+im.shape)
    im.shape

    pred = model.predict(im)

    p = pred.copy()
    p = p.reshape(p.shape[1:-1])

    p[np.where(p>.2)] = 1
    p[np.where(p<.2)] = 0

    im = io.imread(imgpath)
    im = cv2.resize(im,img_size)
    im = np.array(im)

    im[:,:,0] = im[:,:,0]*p 
    im[:,:,0][np.where(p!=1)] = 247
    im[:,:,1] = im[:,:,1]*p 
    im[:,:,1][np.where(p!=1)] = 231
    im[:,:,2] = im[:,:,2]*p
    im[:,:,2][np.where(p!=1)] = 230

    plt.subplot(1,3,2)
    plt.imshow(im)

    if maskpath:
        plt.subplot(1,3,3)
        mask = io.imread(maskpath)
        plt.imshow(mask)

        plt.show()

In [44]:
n_images = 5
for i in np.random.randint(0,len(input_img_val_paths_coco),n_images):
      ploting(input_img_val_paths_coco[i], target_img_val_paths_coco[i])